In [3]:
import torch
import sys
from fastmri.data.subsample import create_mask_for_mask_type
sys.path.insert(0, "/srv/share4/ksarangmath3/mri/fastMRI/fastmri/models/")
from fastmri.pl_modules import FastMriDataModule, UnetModule
from fastmri.data.transforms import UnetDataTransform
from fastmri.data import CombinedSliceDataset, SliceDataset
from fastmri.evaluate import ssim
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#LOAD MODEL HERE, THIS SHOULD BE ALL YOU NEED TO CHANGE
model = UnetModule.load_from_checkpoint(checkpoint_path='/srv/share4/ksarangmath3/mri/fastMRI/fastmri_examples/unet/unet/unet_demo/checkpoints/epoch=40-step=87944.ckpt')
model.eval()

challenge = 'singlecoil'
data_partition = 'val'
data_path =  f"/srv/share4/ksarangmath3/mri/data/{challenge}_{data_partition}"
mask = create_mask_for_mask_type(
        'random', [0.08], [4]
    )
val_transform = UnetDataTransform(challenge, mask_func=mask)
dataset = SliceDataset(
                root=data_path,
                transform=val_transform,
                # sample_rate=sample_rate,
                # volume_sample_rate=volume_sample_rate,
                challenge=challenge,
                use_dataset_cache=True
            )
# sampler = torch.utils.data.DistributedSampler(dataset)
dataloader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size=1,
            num_workers=0,
            # worker_init_fn=worker_init_fn,
            # sampler=sampler,
        )
dataset_len = len(dataset)


In [13]:
ssim_list = []

#baseline SSIM means no reconstruction is done on downsampled image when computing SSIM to target
ssim_baseline_list = []
for i,data in enumerate(dataloader):
    with torch.no_grad():
        if i % 100 == 0:
            print(f"done with {i} steps out of {dataset_len} steps")
            print(f"model SSIM is {0 if i == 0 else np.mean(ssim_list)}")
            print(f"baseline SSIM is {0 if i == 0 else np.mean(ssim_baseline_list)}")
        ssim_list.append(ssim(model(data[0]).numpy(), data[1].numpy()))
        ssim_baseline_list.append(ssim(data[0].numpy(), data[1].numpy()))
        if i > 15:
            plt.imsave("original_unet.png",data[0].squeeze(),cmap="gray")
            plt.imsave("target_unet.png",data[1].squeeze(),cmap="gray")
            plt.imsave("reconstruction_unet.png",model(data[0]).squeeze(),cmap="gray")
            break
        # print(ssim_list[-1])

print(np.mean(ssim_list), '- MODEL SSIM')
print(np.mean(ssim_baseline_list), '- BASELINE SSIM')

done with 0 steps out of 1414 steps
model SSIM is 0
baseline SSIM is 0
0.4114609457125574 - MODEL SSIM
0.3001654195007136 - BASELINE SSIM
